In [1]:
import matplotlib.pyplot as plt
import torch
#from sklearn.manifold import TSNE
from torch_geometric.nn import Node2Vec


In [3]:
import pandas as pd

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
nodes_path = 'nodes_network3.parquet'
df_nodes = pd.read_parquet(nodes_path)
df_nodes

,node,emo_pos,emo_neg,emo_anx,emo_anger,emo_sad,moral
0,online,0.853785,0.937901,0.258591,0.183619,0.161160,0.271575
1,recognize,1.037405,1.114885,0.346107,0.197099,0.181450,0.450382
2,inner,1.106880,1.165342,0.304915,0.199316,0.166026,0.463333
3,travel,1.210000,0.863011,0.312688,0.111935,0.155269,0.224946
4,hope,2.423115,1.149344,0.262459,0.224262,0.227705,0.489016
...,...,...,...,...,...,...,...
798,related,0.832292,0.803333,0.313125,0.096667,0.194583,0.246875
799,glad,2.922500,1.478750,0.363750,0.315000,0.286250,0.258750
800,lately,1.179500,1.913000,0.479500,0.287000,0.392000,0.215000
801,plenty,1.032500,0.765250,0.181750,0.140500,0.212250,0.236250


Before continuing, check if we can see morality words with higher score

In [9]:
df_nodes.sort_values(by="moral", ascending=False)[0:50]

,node,emo_pos,emo_neg,emo_anx,emo_anger,emo_sad,moral
441,blame,0.717194,1.227092,0.186480,0.301071,0.158316,1.849898
50,shame,0.880000,2.996316,0.363743,0.242222,0.312632,1.799123
593,judge,1.011523,1.335273,0.488242,0.237461,0.119727,1.715547
564,fault,0.923898,1.166864,0.141271,0.293390,0.213559,1.660424
286,deserve,1.645598,1.430726,0.256410,0.308034,0.211068,1.562735
309,wrong,0.846250,1.251149,0.257027,0.302128,0.160878,1.526014
537,fake,1.415152,1.273939,0.251515,0.340909,0.158788,1.510303
33,lazy,0.893825,1.141257,0.260820,0.221694,0.199781,1.326230
311,excuse,0.680070,1.027413,0.254336,0.178252,0.159161,1.312308
287,useful,0.953398,0.760291,0.244854,0.099903,0.098058,1.304466


In [6]:
edges_path = 'edges_network3.parquet'
df_edges = pd.read_parquet(edges_path)
df_edges

,node1_norm,node2_norm,weight
0,ever,recently,3
1,eating,parent,6
2,drink,guess,4
3,die,waste,8
4,guess,weed,4
...,...,...,...
218865,bring,crazy,1
218866,free,voice,1
218867,group,search,1
218868,hope,journaling,1


Explore pairs of words with more weight

In [10]:
df_edges.sort_values(by="weight", ascending=False)[0:50]

,node1_norm,node2_norm,weight
194408,man,woman,773
163907,book,read,542
199706,game,play,436
44885,media,social,316
137700,game,video,289
64196,high,school,271
159570,bed,sleep,255
23048,sex,woman,238
30229,drink,water,233
173534,smoke,weed,229


Create graph from data